# Audio Classification Training on Colab

This notebook sets up and runs training for Mamba, Liquid S4, and V-JEPA2 models on ESC-50 dataset.

## Quick Start
1. Enable GPU: Runtime → Change runtime type → GPU (T4 or V100)
2. Run all cells below
3. Choose which model to train (Mamba, Liquid S4, or V-JEPA2)


In [ ]:
# Install required packages
!pip install torch torchaudio librosa pandas numpy matplotlib seaborn scikit-learn tqdm wandb einops transformers

# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected! Please enable GPU: Runtime → Change runtime type → GPU")


In [ ]:
# Clone your repository (replace with your actual GitHub repo URL)
# Option 1: Public repo (no authentication needed)
!git clone https://github.com/YOUR_USERNAME/audio-classifier.git

# Option 2: Private repo (requires GitHub token)
# !git clone https://YOUR_GITHUB_TOKEN@github.com/YOUR_USERNAME/audio-classifier.git

# Change to project directory
import os
os.chdir('audio-classifier')

# Initialize git submodules (for external models)
!git submodule update --init --recursive

print("✅ Repository cloned and submodules initialized!")


In [ ]:
# Download ESC-50 dataset if not present
import os
if not os.path.exists('data/ESC-50'):
    print("📥 Downloading ESC-50 dataset...")
    !mkdir -p data
    !cd data && wget -q https://github.com/karolpiczak/ESC-50/archive/master.zip
    !cd data && unzip -q master.zip && mv ESC-50-master ESC-50
    !cd data && rm master.zip
    print("✅ ESC-50 dataset downloaded successfully!")
else:
    print("✅ ESC-50 dataset already exists!")
    
# Check dataset
import pandas as pd
meta_df = pd.read_csv('data/ESC-50/meta/esc50.csv')
print(f"📊 Dataset info: {len(meta_df)} samples, {len(meta_df['category'].unique())} classes")
print(f"🎵 Categories: {', '.join(meta_df['category'].unique()[:10])}...")


In [ ]:
# Test imports and model creation
import sys
sys.path.append('.')

from src.models.mamba_audio import MambaAudioClassifier
from src.models.liquidS4_audio import LiquidS4AudioClassifier
from src.models.vjepa_audio import VJEPA2AudioClassifier
from configs.model_configs import get_mamba_config, get_liquid_s4_config, get_vjepa2_config

print("✅ All imports successful!")

# Test model creation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Using device: {device}")

# Test Mamba model
print("\n🧪 Testing Mamba model...")
mamba_config = get_mamba_config()
mamba_model = MambaAudioClassifier(**mamba_config, device=device)
mamba_params = sum(p.numel() for p in mamba_model.parameters())
print(f"   Mamba: {mamba_params:,} parameters")

# Test Liquid S4 model
print("🧪 Testing Liquid S4 model...")
s4_config = get_liquid_s4_config()
s4_model = LiquidS4AudioClassifier(**s4_config, device=device)
s4_params = sum(p.numel() for p in s4_model.parameters())
print(f"   Liquid S4: {s4_params:,} parameters")

# Test V-JEPA2 model
print("🧪 Testing V-JEPA2 model...")
vjepa_config = get_vjepa2_config()
vjepa_model = VJEPA2AudioClassifier(**vjepa_config)
vjepa_params = sum(p.numel() for p in vjepa_model.parameters())
print(f"   V-JEPA2: {vjepa_params:,} parameters")

print("\n✅ All models created successfully!")


## 🚀 Training Options

Choose which model to train by running one of the cells below:


In [ ]:
# 🎯 Train Mamba Model (Recommended for Colab - fastest training)
print("🚀 Starting Mamba training...")
!python scripts/cloud_train.py --model mamba --batch_size 16 --epochs 50 --lr 0.001

print("✅ Mamba training completed!")


In [ ]:
# 🎯 Train Liquid S4 Model
print("🚀 Starting Liquid S4 training...")
!python scripts/cloud_train.py --model liquid_s4 --batch_size 16 --epochs 50 --lr 0.001

print("✅ Liquid S4 training completed!")


In [ ]:
# 🎯 Train V-JEPA2 Model (May require more memory)
print("🚀 Starting V-JEPA2 training...")
!python scripts/cloud_train.py --model vjepa2 --batch_size 8 --epochs 50 --lr 0.001

print("✅ V-JEPA2 training completed!")


In [ ]:
# 📥 Download trained models and results
from google.colab import files
import os

print("📦 Preparing downloads...")

# Download checkpoints
if os.path.exists('checkpoints'):
    print("📁 Zipping checkpoints...")
    !zip -r checkpoints.zip checkpoints/
    files.download('checkpoints.zip')
    print("✅ Checkpoints downloaded!")

# Download results/logs if they exist
if os.path.exists('results'):
    print("📁 Zipping results...")
    !zip -r results.zip results/
    files.download('results.zip')
    print("✅ Results downloaded!")

print("🎉 All downloads completed!")
